In [1]:
import ollama

In [6]:
print(dir(ollama))

['AsyncClient', 'ChatResponse', 'Client', 'EmbedResponse', 'EmbeddingsResponse', 'GenerateResponse', 'Image', 'ListResponse', 'Message', 'Options', 'ProcessResponse', 'ProgressResponse', 'RequestError', 'ResponseError', 'ShowResponse', 'StatusResponse', 'Tool', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_client', '_types', '_utils', 'chat', 'copy', 'create', 'delete', 'embed', 'embeddings', 'generate', 'list', 'ps', 'pull', 'push', 'show']


In [12]:
question = 'How can LLMs be used in engineering?'
response = ollama.generate(model='gemma3:4b', prompt=question)
print(response['response'])

Large Language Models (LLMs) are rapidly finding their way into various engineering disciplines, offering exciting possibilities for automation, efficiency, and innovation. Here's a breakdown of how they're being used, categorized by engineering field and function:

**1. General Engineering Applications (Across Disciplines):**

* **Documentation & Knowledge Management:**
    * **Generating Technical Reports:** LLMs can draft initial drafts of reports, specifications, and proposals based on prompts and data.
    * **Creating Standard Operating Procedures (SOPs):**  They can generate SOPs for equipment operation, maintenance, and troubleshooting.
    * **Knowledge Base Creation & Retrieval:**  LLMs can synthesize information from vast amounts of technical documentation, research papers, and internal knowledge bases, making it easier to find relevant information.
    * **Translation:**  Quickly translate technical documents between languages.
* **Code Generation & Assistance:**
    * **Ge

In [15]:
import ollama
import json
import re
import sqlite3
from datetime import datetime
import os

class OllamaProductExtractor:
    """Product and price extractor using a local Ollama model"""

    def __init__(self, model_name: str = 'gemma3:4b'):
        """Initialize the Ollama-based product extractor"""
        self.model_name = model_name
        self.setup_database()

    def adapt_datetime(dt):
        return dt.isoformat()

    def convert_datetime(bytestring):
        return datetime.fromisoformat(bytestring.decode())

    sqlite3.register_adapter(datetime, adapt_datetime)
    sqlite3.register_converter('DATETIME', convert_datetime)

    def setup_database(self):
        """Setup SQLite database for storing products"""
        self.conn = sqlite3.connect('products_ollama_gemma.db', detect_types=sqlite3.PARSE_DECLTYPES)
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY,
            name TEXT,
            price REAL,
            currency TEXT,
            source_text TEXT,
            timestamp DATETIME
        )
        ''')
        self.conn.commit()

    def extract_products(self, text: str) -> list[dict[str, object]]:
        """Extract product names and prices from text using Ollama"""
        system_prompt = """
        Extract all products and their prices mentioned in the text.
        Return a JSON array where each item has the following format:
        {
            "name": "Product Name",
            "price": 123.45,
            "currency": "$"
        }

        Rules:
        1. Extract complete product names including brand and model
        2. Convert all prices to numeric values (no currency symbols in the price field)
        3. Identify the currency symbol used ($, €, £, etc.) and include it separately
        4. Return an empty array if no products with prices are detected
        5. Do not make up any information not present in the text
        6. Extract bangla text as well
        """

        try:
            response = ollama.chat(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": text}
                ]
            )

            response_text = response['message']['content']

            # Parse the response JSON
            try:
                # Try direct JSON parsing
                result = json.loads(response_text)

                # Add source text to each product entry
                for product in result:
                    product["source_text"] = text

                return result

            except json.JSONDecodeError:
                # Fallback if response isn't valid JSON - look for code blocks
                json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
                if json_match:
                    try:
                        result = json.loads(json_match.group(1))
                        for product in result:
                            product["source_text"] = text
                        return result
                    except:
                        pass

                print(f"Failed to parse Ollama response as JSON: {response_text}")
                return []

        except Exception as e:
            print(f"Error calling Ollama API: {e}")
            return []

    def store_products(self, products: list[dict[str, object]]):
        """Store extracted products in the database"""
        for product in products:
            self.cursor.execute(
                "INSERT INTO products (name, price, currency, source_text, timestamp) VALUES (?, ?, ?, ?, ?)",
                (product["name"],
                 product["price"],
                 product["currency"],
                 product["source_text"],
                 datetime.now())
            )
        self.conn.commit()

    def process_text(self, text: str) -> list[dict[str, object]]:
        """Process text to extract and store products"""
        products = self.extract_products(text)
        if products:
            self.store_products(products)
        return products

    def process_voice(self, audio_file_path: str):
        """
        Placeholder for voice processing functionality
        In a real implementation, this would use a speech-to-text API
        then pass the text to process_text()
        """
        print(f"Voice processing not implemented in this demo. Would process: {audio_file_path}")
        return []

    def get_stored_products(self, limit: int = 10) -> list[dict[str, object]]:
        """Retrieve stored products from database"""
        self.cursor.execute(
            "SELECT id, name, price, currency, source_text, timestamp FROM products ORDER BY timestamp DESC LIMIT ?",
            (limit,)
        )
        columns = ["id", "name", "price", "currency", "source_text", "timestamp"]
        return [dict(zip(columns, row)) for row in self.cursor.fetchall()]

    def close(self):
        """Close database connection"""
        self.conn.close()


# Demo usage
def run_ollama_demo():
    # Initialize the extractor
    try:
        extractor = OllamaProductExtractor()

        # Sample texts
        sample_texts = [
            "amare 5kg chal den 1000 taka",
            "I bought a Samsung Galaxy S23 for $999 and AirPods Pro for $249"
        ]

        # Process each sample text
        all_products = []
        for text in sample_texts:
            print(f"\nProcessing text: {text}")
            products = extractor.process_text(text)
            all_products.extend(products)
            print(f"Extracted products: {json.dumps(products, indent=2)}")

        # Show all stored products
        stored_products = extractor.get_stored_products()
        print("\nStored products in database:")
        for product in stored_products:
            print(
                f"ID: {product['id']} | {product['name']} - {product['currency']}{product['price']} | Source: '{product['source_text'][:30]}...'")

        # Close the connection
        extractor.close()
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have Ollama installed and running with the appropriate model.")


if __name__ == "__main__":
    run_ollama_demo()


Processing text: amare 5kg chal den 1000 taka
Extracted products: [
  {
    "name": "5kg chal",
    "price": 1000.0,
    "currency": "\u09f3",
    "source_text": "amare 5kg chal den 1000 taka"
  }
]

Processing text: I bought a Samsung Galaxy S23 for $999 and AirPods Pro for $249
Extracted products: [
  {
    "name": "Samsung Galaxy S23",
    "price": 999.0,
    "currency": "$",
    "source_text": "I bought a Samsung Galaxy S23 for $999 and AirPods Pro for $249"
  },
  {
    "name": "AirPods Pro",
    "price": 249.0,
    "currency": "$",
    "source_text": "I bought a Samsung Galaxy S23 for $999 and AirPods Pro for $249"
  }
]

Stored products in database:
ID: 3 | AirPods Pro - $249.0 | Source: 'I bought a Samsung Galaxy S23 ...'
ID: 2 | Samsung Galaxy S23 - $999.0 | Source: 'I bought a Samsung Galaxy S23 ...'
ID: 1 | 5kg chal - ৳1000.0 | Source: 'amare 5kg chal den 1000 taka...'


# Deepseek

In [14]:
import ollama
import json
import re
import sqlite3
from datetime import datetime
import os

class OllamaProductExtractor:
    """Product and price extractor using a local Ollama model"""

    def __init__(self, model_name: str = 'deepseek-r1:1.5b'):
        """Initialize the Ollama-based product extractor"""
        self.model_name = model_name
        self.setup_database()

    def adapt_datetime(dt):
        return dt.isoformat()

    def convert_datetime(bytestring):
        return datetime.fromisoformat(bytestring.decode())

    sqlite3.register_adapter(datetime, adapt_datetime)
    sqlite3.register_converter('DATETIME', convert_datetime)

    def setup_database(self):
        """Setup SQLite database for storing products"""
        self.conn = sqlite3.connect('products_ollama_deepseek.db', detect_types=sqlite3.PARSE_DECLTYPES)
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY,
            name TEXT,
            price REAL,
            currency TEXT,
            source_text TEXT,
            timestamp DATETIME
        )
        ''')
        self.conn.commit()

    def extract_products(self, text: str) -> list[dict[str, object]]:
        """Extract product names and prices from text using Ollama"""
        system_prompt = """
        Extract all products and their prices mentioned in the text.
        Return a JSON array where each item has the following format:
        {
            "name": "Product Name",
            "price": 123.45,
            "currency": "$"
        }

        Rules:
        1. Extract complete product names including brand and model
        2. Convert all prices to numeric values (no currency symbols in the price field)
        3. Identify the currency symbol used ($, €, £, etc.) and include it separately
        4. Return an empty array if no products with prices are detected
        5. Do not make up any information not present in the text
        6. Extract bangla text as well
        """

        try:
            response = ollama.chat(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": text}
                ]
            )

            response_text = response['message']['content']

            # Parse the response JSON
            try:
                # Try direct JSON parsing
                result = json.loads(response_text)

                # Add source text to each product entry
                for product in result:
                    product["source_text"] = text

                return result

            except json.JSONDecodeError:
                # Fallback if response isn't valid JSON - look for code blocks
                json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
                if json_match:
                    try:
                        result = json.loads(json_match.group(1))
                        for product in result:
                            product["source_text"] = text
                        return result
                    except:
                        pass

                print(f"Failed to parse Ollama response as JSON: {response_text}")
                return []

        except Exception as e:
            print(f"Error calling Ollama API: {e}")
            return []

    def store_products(self, products: list[dict[str, object]]):
        """Store extracted products in the database"""
        for product in products:
            self.cursor.execute(
                "INSERT INTO products (name, price, currency, source_text, timestamp) VALUES (?, ?, ?, ?, ?)",
                (product["name"],
                 product["price"],
                 product["currency"],
                 product["source_text"],
                 datetime.now())
            )
        self.conn.commit()

    def process_text(self, text: str) -> list[dict[str, object]]:
        """Process text to extract and store products"""
        products = self.extract_products(text)
        if products:
            self.store_products(products)
        return products

    def process_voice(self, audio_file_path: str):
        """
        Placeholder for voice processing functionality
        In a real implementation, this would use a speech-to-text API
        then pass the text to process_text()
        """
        print(f"Voice processing not implemented in this demo. Would process: {audio_file_path}")
        return []

    def get_stored_products(self, limit: int = 10) -> list[dict[str, object]]:
        """Retrieve stored products from database"""
        self.cursor.execute(
            "SELECT id, name, price, currency, source_text, timestamp FROM products ORDER BY timestamp DESC LIMIT ?",
            (limit,)
        )
        columns = ["id", "name", "price", "currency", "source_text", "timestamp"]
        return [dict(zip(columns, row)) for row in self.cursor.fetchall()]

    def close(self):
        """Close database connection"""
        self.conn.close()


# Demo usage
def run_ollama_demo():
    # Initialize the extractor
    try:
        extractor = OllamaProductExtractor()

        # Sample texts
        sample_texts = [
            "amare 5kg chal den 1000 taka",
            "I bought a Samsung Galaxy S23 for $999 and AirPods Pro for $249"
        ]

        # Process each sample text
        all_products = []
        for text in sample_texts:
            print(f"\nProcessing text: {text}")
            products = extractor.process_text(text)
            all_products.extend(products)
            print(f"Extracted products: {json.dumps(products, indent=2)}")

        # Show all stored products
        stored_products = extractor.get_stored_products()
        print("\nStored products in database:")
        for product in stored_products:
            print(
                f"ID: {product['id']} | {product['name']} - {product['currency']}{product['price']} | Source: '{product['source_text'][:30]}...'")

        # Close the connection
        extractor.close()
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have Ollama installed and running with the appropriate model.")


if __name__ == "__main__":
    run_ollama_demo()


Processing text: amare 5kg chal den 1000 taka
Failed to parse Ollama response as JSON: <think>
Alright, let me try to figure out how to approach this problem. So, the user has given a query where they want all products and their prices extracted in a specific JSON format. They also mentioned that the text includes bangla (Assam) content, so I need to make sure that my solution is both accurate and language-agnostic.

First, I'll break down the requirements. The output should be an array of objects, each containing "name", "price", and "currency". The price needs to be converted into a numeric format without any currency symbols, but the currency itself must be included as part of the object. If no products are found, it should return an empty array.

Looking at the example provided: "amare 5kg chal den 1000 taka". I notice that this is in Bangla. The name here is "5kg chal" and the price is "1000 taka". So, my solution needs to handle different languages correctly.

I'll start by pars